In [13]:
import numpy as np
#1. define categories and Questions
categories = ["Exoplanets", "Galaxies", "Constellations", "Planets", "Moons", "Astronomers/Physicists/Astronomers"]
# 5 rows = 
board_values = np.array([[100, 100, 100, 100, 100, 100], 
                         [200, 200, 200, 200, 200, 200], 
                         [300, 300, 300, 300, 300, 300], 
                         [400, 400, 400, 400, 400, 400], 
                         [500, 500, 500, 500, 500, 500] ])

# 2. Randomly pick a Daily Double location using choice
dd_selection = np.random.choice(30)

# Convert that one number into specific row (0-4) and col (0-5)
dd_row = dd_selection // 6
dd_col = dd_selection % 6

print(f"Daily Double Location: Row {dd_row}, Col {dd_col} (Category: {categories[dd_col]})")

# 3. Initialize Board with placeholders (5 rows x 6 columns)

ddArray = np.array(["Q" for _ in range(30)])
ddArray = ddArray.reshape(5, 6)
ddArray[dd_row][dd_col] = "DD" # Mark Daily Double

# 4. Display Board
print("\nJeopardy Board:")
print(ddArray)

list = []
for i in range(30):
    list.append(False)
beenUsed = np.array(list)
beenUsed = beenUsed.reshape(5, 6)
print(beenUsed)

Daily Double Location: Row 0, Col 2 (Category: Constellations)

Jeopardy Board:
[['Q' 'Q' 'D' 'Q' 'Q' 'Q']
 ['Q' 'Q' 'Q' 'Q' 'Q' 'Q']
 ['Q' 'Q' 'Q' 'Q' 'Q' 'Q']
 ['Q' 'Q' 'Q' 'Q' 'Q' 'Q']
 ['Q' 'Q' 'Q' 'Q' 'Q' 'Q']]
[[False False False False False False]
 [False False False False False False]
 [False False False False False False]
 [False False False False False False]
 [False False False False False False]]


In [ ]:
from clue_object_setup import Tile
from clue_object_setup import generate_tiles
from AI_Setup import create_clues
from AI_Setup import create_answers

answer = create_answers()
clues, answers = create_clues(answer)

generate_tiles(clues, answers, ddArray, board_values, beenUsed)



['This Hot Jupiter exoplanet is infamous for its sideways rain of molten glass and an atmosphere rich in hydrogen sulfide, giving it a foul odor reminiscent of rotten eggs; it was discovered using the radial velocity method.', 'This galaxy, home to our solar system, stretches over 100,000 light-years and requires about 250 million years for our solar system to complete one orbit around it.', 'This constellation, the third largest in the sky and known as the Great Bear, represents Callisto, a nymph loved by Zeus in Greek mythology.', 'This distant blue planet, first glimpsed by Galileo in 1612 and mathematically predicted in 1846, has days lasting about 16 hours and seasons that stretch over four decades due to its lengthy orbit.', 'This moon, second largest in our solar system and unique for its dense atmosphere, experiences a rain-like cycle similar to Earth’s.', 'This 1967 Air Force officer, the first black astronaut, earned a PhD in Physical Chemistry from Ohio State University befo

ValueError: too many values to unpack (expected 2)